In [1]:
import tensorflow as tf
import keras
#from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Model
from keras.layers import *
import os
import sys
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir = "C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/score_pics/"
train_dir = base_dir + "train_pics/"
validation_dir = base_dir + "validation_pics/"
test_dir = base_dir + "test_pics/"

In [3]:
print("total training images:", len(os.listdir(train_dir)))
print("total validation images:", len(os.listdir(validation_dir)))
print("total test images:", len(os.listdir(test_dir)))



total training images: 29
total validation images: 14
total test images: 16


## Keras Sequential API (old)

In [31]:
model = ks.models.Sequential()
model.add(layers.Conv2D(16, (5,5), activation = "relu", input_shape=(160, 50, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32, (5,5), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 156, 46, 16)       1216      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 78, 23, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 74, 19, 32)        12832     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 37, 9, 32)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 33, 5, 64)         51264     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 2, 64)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2048)              0         
Total para

In [ ]:
model.compile(loss="binary_crossentropy",
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

In [35]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(160, 50),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(160, 75),
        batch_size=20,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(160, 75),
        batch_size=20,
        class_mode='binary')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


## Keras functional API

In [75]:
train_folder = base_dir + "train_pics"
valid_folder = base_dir + "validation_pics"
test_folder = base_dir + "test_pics"

In [76]:
train_names = [f for f in os.listdir(train_folder)]
valid_names = [f for f in os.listdir(valid_folder)]
test_names = [f for f in os.listdir(test_folder)]

In [77]:
print(str(len(train_names)) + " training files")
print(str(len(valid_names)) + " validation files")
print(str(len(test_names)) + " test files")

29 training files
13 validation files
15 test files


In [127]:
def create_dataset(fold, names):
    dataset = np.ndarray(shape=(len(names), 50, 160, 3), dtype = np.float32)
    i = 0
    for f in names:
        img = load_img(base_dir + fold + "/" + f)
        x = img_to_array(img)
        #x = x.reshape(())
        x = (x - 128) / 128
        dataset[i] = x
        i += 1
    return dataset

def create_labels(names):
    labels = [np.zeros((len(names),10)) for i in range(5)]
    i = 0
    for n in range(len(names)):
        seq = names[n][0:5]
        for digit in range(len(seq)):
            label = int(seq[digit])
            #print(str(digit) + ";" + str(label) + ";" + str(n))
            labels[digit][n][label] = 1
        i += 1
    return labels

In [128]:
[np.array((len(train_names),10)) for i in range(5)]

[array([29, 10]),
 array([29, 10]),
 array([29, 10]),
 array([29, 10]),
 array([29, 10])]

In [129]:
train_dataset = create_dataset("train_pics", train_names)
validation_dataset = create_dataset("validation_pics", valid_names)
test_dataset = create_dataset("test_pics", test_names)

train_labels = create_labels(train_names)
validation_labels = create_labels(valid_names)
test_dataset = create_labels(test_names)

In [130]:
train_labels[0:1]

[array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [94]:
train_labels

chararray([b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
           b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0',
           b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0', b'0'],
          dtype='|S1')

In [39]:
#Filters:
f1, f2, f3 = 16,32,64
patch_size = 5
k_size = (3,3)
p_size = (2,2)
n_class = 10
node = 10

#TODO correct shape
inp = Input(shape=(75, 250, 1))


conv1 = Conv2D(filters = f1, kernel_size = k_size, activation = "relu")(inp)
maxp1 = MaxPool2D(pool_size = p_size)(conv1)

conv2 = Conv2D(filters = f2, kernel_size = k_size, activation = "relu")(maxp1)
maxp2 = MaxPool2D(pool_size = p_size)(conv2)

conv3 = Conv2D(filters = f3, kernel_size = k_size, activation = "relu")(maxp2)
maxp3 = MaxPool2D(pool_size = p_size)(conv3)

flat = Flatten()(maxp3)
drop = Dropout(0.25)(flat)

dense_11 = Dense(node, activation = "relu")(drop)
dense_12 = Dense(node, activation = "relu")(drop)
dense_13 = Dense(node, activation = "relu")(drop)
dense_14 = Dense(node, activation = "relu")(drop)
dense_15 = Dense(node, activation = "relu")(drop)

dense_21 = Dense(n_class, activation = "softmax")(dense_11)
dense_22 = Dense(n_class, activation = "softmax")(dense_12)
dense_23 = Dense(n_class, activation = "softmax")(dense_13)
dense_24 = Dense(n_class, activation = "softmax")(dense_14)
dense_25 = Dense(n_class, activation = "softmax")(dense_15)

output = [dense_21, dense_22, dense_23, dense_24, dense_25]


model = Model(input = inp, output = output)
model.compile(loss = "categorical_crossentropy", optimizer = "adadelta", metrics = ["accuracy"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [40]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 75, 250, 1)   0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 73, 248, 16)  160         input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_28 (MaxPooling2D) (None, 36, 124, 16)  0           conv2d_28[0][0]                  
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 34, 122, 32)  4640        max_pooling2d_28[0][0]           
__________________________________________________________________________________________________
max_poolin